In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Fri Jan  3 15:53:07 2020

@author: whitma01
"""
import cv2
import numpy as np
from PIL import Image
import sklearn.metrics as sk
from tqdm import tqdm
import os
import SimpleITK as sitk
import glob, os

def remove_small_regions(m_mask, threshold): 

    img = sitk.GetImageFromArray(m_mask)
    cca = sitk.ConnectedComponentImageFilter()
    cca_image = cca.Execute(img)
    stats = sitk.LabelShapeStatisticsImageFilter()                                                                                                                                     
    stats.Execute(cca_image)

    relabelMap =  { i : 0 for i in stats.GetLabels() if stats.GetNumberOfPixels(i) < threshold }
    output = sitk.ChangeLabel(cca_image, changeMap=relabelMap)

    del img
    del cca
    del cca_image
    del stats

    m_mask_inverse = sitk.GetArrayFromImage(output)
    m_mask_inverse = m_mask_inverse != 0
    return m_mask_inverse

#https://biomedical-engineering-online.biomedcentral.com/articles/10.1186/s12938-018-0518-0


exclude = ["685039_(1.00,17661,41360,19968,2561)",
    "685039_(1.00,7007,741,9216,1024)",
    "685039_(1.00,81054,80,2561,3585)",
    "685040_(1.00,12444,41415,17920,1536)",
    "685042_(1.00,14,40462,17409,1025)",
    "685042_(1.00,22058,190,12800,1024)",
    "688058_(1.00,15734,32438,3072,2560)",
    "688059_(1.00,22312,137,2560,2049)",
    "688059_(1.00,30214,40324,6656,1537)",
    "688060_(1.00,2000,40,9728,2048)",
    "688061_(1.00,11308,19,6656,513)",
    "688061_(1.00,3687,38203,10240,3072)",
    "688062_(1.00,2057,40517,2561,1537)",
    "688062_(1.00,29137,99,3072,1537)",
    "688063_(1.00,144,39239,8192,1536)",
    "688063_(1.00,59467,8417,1024,3584)",
    "688064_(1.00,36,16442,5120,2560)",
    "688065_(1.00,10100,27089,1024,1024)",
    "688065_(1.00,17683,18489,1024,2048)",
    "688065_(1.00,70846,38229,7168,2560)",
    "688066_(1.00,19682,78,2048,2561)",
    "688066_(1.00,37781,26018,1025,1025)",
    "688067_(1.00,36205,37462,2048,1024)",
    "688067_(1.00,36818,33694,1025,1025)",
    "688067_(1.00,6355,8220,3072,2560)",
    "688068_(1.00,29035,40904,3072,2560)",
    "688068_(1.00,68661,18472,3072,1024)",
    "688069_(1.00,29,3205,513,4096)",
    "688070_(1.00,39203,33602,2048,1024)",
    "688070_(1.00,42838,10,1536,2560)",
    "688071_(1.00,48355,40346,3584,1536)",
    "688071_(1.00,68742,11843,1536,1536)",
    "688072_(1.00,167,29474,4608,1536)",
    "688072_(1.00,37596,7981,2560,2048)",
    "688072_(1.00,43334,28816,2048,2560)",
    "688072_(1.00,51839,26482,1536,1536)",
    "688073_(1.00,41390,5217,1536,1536)",
    "688073_(1.00,42068,31365,2048,4096)",
    "688073_(1.00,6443,2178,1025,4097)",
    "688074_(1.00,5212,37585,3584,1024)",
    "688074_(1.00,6985,33897,1536,1024)",
    "688075_(1.00,52,24706,3585,2048)",
    "688076_(1.00,2373,8304,1024,2048)",
    "688076_(1.00,40252,19178,2048,3072)",
    "688076_(1.00,998,13583,1025,1025)",
    "688077_(1.00,19589,3339,1024,1024)",
    "688077_(1.00,43727,34903,2560,512)",
    "688077_(1.00,464,357,3072,1024)",
    "688077_(1.00,68948,32390,1536,512)",
    "688078_(1.00,44049,40715,5120,2560)",
    "688078_(1.00,63258,7450,1536,1024)",
    "688078_(1.00,65548,21132,2049,1025)",
    "688079_(1.00,23762,5020,1024,2048)",
    "688079_(1.00,4134,17635,1024,2560)",
    "688079_(1.00,46337,3527,1024,1536)",
    "688080_(1.00,2964,22951,2048,1536)",
    "688080_(1.00,36554,385,1024,1536)",
    "688081_(1.00,23205,154,8192,512)",
    "688081_(1.00,5887,36485,2048,2048)",
    "688082_(1.00,34697,37863,4608,3584)",
    "688083_(1.00,33758,10111,2048,1536)",
    "688084_(1.00,31634,38140,3072,2560)",
    "688135_(1.00,15442,3051,3072,1024)",
    "688135_(1.00,55423,30392,1024,2048)",
    "688085_(1.00,13134,1089,14848,4096)",]



# names = ['684913','684914','684916','685039','685040','685041','685042','685281']
# directory_true = 'masksMISSION_corrected/'
# directory_pred = 'Inference_MISSION/'
# data_writer = open("evaluation/validate_MISSION_together.csv", "w")

names = ['688058','688059','688060','688061','688062','688063','688064','688065',
         '688066','688067','688068','688069','688070','688071','688072','688073','688074',
         '688075','688076','688077','688078','688079','688080','688081','688082','688083','688084','688135']
directory_true = 'masksBEHOLD_corrected/'
directory_pred = 'Inference_BEHOLD/'
data_writer = open("evaluation/validate_BEHOLD_together.csv", "w")


data_writer.write("Name, TruePositives, FalsePositives, FalseNegatives, Precision, Recall, F1score, split, merge, total predicted, total true\n")
data_writer.flush()
os.fsync(data_writer.fileno())

TruePositives_total = 0
FalsePositives_total = 0
FalseNegatives_total = 0
split_total = 0
merge_total = 0
nr_vessels_true = 0
nr_predicted_true = 0

for i, name in enumerate(names):
    print(name)
    print() 

    for file in glob.glob(directory_pred + "*.png"):
        
        if name in file and 'Vessel' in file:
            
            imageID = os.path.basename(file[:-4])
            patientID = imageID.split("_")
            imageID2 = patientID[0] + "_" + patientID[1]

            if imageID2 in exclude:
                print("excluding " + imageID2)
            else:

                TruePositives = 0
                FalsePositives = 0
                FalseNegatives = 0
                split = 0
                merge = 0



                imageID = os.path.basename(file[:-4])
                imageID2 = imageID.split("_")

                filename_predicted = directory_pred + os.path.basename(file)
                print('reading predicted mask ' + filename_predicted)
                mask_predicted = cv2.imread(filename_predicted, 0)

                filename_true = directory_true + imageID2[0] + "_vessel_" + imageID2[1] + "-mask.png"
                print('reading true mask ' + filename_true)
                mask_true = cv2.imread(filename_true, 0)

                
                mask_predicted = remove_small_regions(mask_predicted, 40)
                mask_predicted = np.uint8(mask_predicted)
                mask_true = remove_small_regions(mask_true, 40)
                mask_true = np.uint8(mask_true)


                print('calculating connected components')
                mask_predicted = cv2.threshold(mask_predicted, mask_predicted.min(), mask_predicted.max(), cv2.THRESH_BINARY)[1]  # ensure binary
                mask_true = cv2.threshold(mask_true, mask_true.min(), mask_true.max(), cv2.THRESH_BINARY)[1]  # ensure binary
                nlabels_predicted, labels_predicted, stats_predicted, centroids_predicted  = cv2.connectedComponentsWithStats(mask_predicted, connectivity=8)
                nlabels_true, labels_true, stats_true, centroids_true  = cv2.connectedComponentsWithStats(mask_true, connectivity=8)

                nlabels_predicted = nlabels_predicted - 1
                nlabels_true = nlabels_true - 1

                print('nlabels_predicted: {:.2f}'.format(nlabels_predicted))
                print('nlabels_true: {:.2f}'.format(nlabels_true))

                cv2.imwrite('labels_predicted.jpg', labels_predicted)
                cv2.imwrite('labels_true.jpg', labels_true)


                lookup_table = np.zeros((nlabels_predicted, nlabels_true))
                print('lookup_table.shape: {}'.format(lookup_table.shape))


                height, width = mask_predicted.shape[:2]

                print('height: {} width: {}'.format(height, width))
                print('filling lookup table')
                for x in range(0, width):
                    for y in range(0, height):
                        i = labels_predicted[y,x]
                        j = labels_true[y,x]
                        if i != 0 and j != 0:
                            lookup_table[i-1,j-1] += 1


                size_list_predicted = np.zeros(nlabels_predicted)
                for x in range(0, width):
                    for y in range(0, height):
                        i = labels_predicted[y,x]
                        if i > 0:
                            size_list_predicted[i-1] += 1

                size_list_true = np.zeros(nlabels_true)
                for x in range(0, width):
                    for y in range(0, height):
                        i = labels_true[y,x]
                        if i > 0:
                            size_list_true[i-1] += 1                


                for i in range(0, nlabels_predicted):
                    total = 0
                    for j in range(0, nlabels_true):
                        total += lookup_table[i,j]
                    if total / size_list_predicted[i] > 0.5:
                        TruePositives += 1
                    else:
                        FalsePositives += 1
                print('TruePositives: {}'.format(TruePositives))
                print('FalsePositives: {}'.format(FalsePositives))
                print('FalseNegatives: {}'.format(FalseNegatives))


                for j in range(0, nlabels_true):
                    total = 0
                    for i in range(0, nlabels_predicted):
                        total += lookup_table[i,j]
                    if total / size_list_true[j] < 0.5:
                        FalseNegatives = FalseNegatives + 1


                print('calculate split or merged')
                for j in range(0, nlabels_true):
                    count = 0
                    for i in range(0, nlabels_predicted):
                        if lookup_table[i,j] != 0:
                            count = count + 1
                    if count > 1:
                        split = split + 1

                for i in range(0, nlabels_predicted):
                    count = 0
                    for j in range(0, nlabels_true):
                        if lookup_table[i,j] != 0:
                            count = count + 1
                    if count > 1:
                        merge = merge + 1

                print('split: {}'.format(split))
                print('merge: {}'.format(merge))


                TruePositives_total += TruePositives
                FalsePositives_total += FalsePositives
                FalseNegatives_total += FalseNegatives
                split_total += split
                merge_total += merge
                nr_predicted_true += nlabels_predicted
                nr_vessels_true += nlabels_true

                print()    


print('TruePositives: {}'.format(TruePositives_total))
print('FalsePositives: {}'.format(FalsePositives_total))
print('FalseNegatives: {}'.format(FalseNegatives_total)) 


Precision = TruePositives_total / (TruePositives_total + FalsePositives_total)
Recall = TruePositives_total / (TruePositives_total + FalseNegatives_total)
F1score = (2 * TruePositives_total) / ((2 * TruePositives_total) + FalsePositives_total + FalseNegatives_total)
print('Precision: {:.2f}'.format(Precision))
print('Recall: {:.2f}'.format(Recall))
print('F1score: {:.2f}'.format(F1score))

print('split: {}'.format(split_total))
print('merge: {}'.format(merge_total))
print()

data_writer.write(name + "," + str(TruePositives_total) + "," + str(FalsePositives_total) + "," + 
                  str(FalseNegatives_total) + "," + str(Precision) + "," + str(Recall) + "," + str(F1score) + 
                  "," + str(split_total) + "," + str(merge_total) + "," + str(nr_predicted_true) + "," + str(nr_vessels_true) + "\n")
data_writer.flush()
os.fsync(data_writer.fileno())

data_writer.close()

688058

excluding 688058_(1.00,15734,32438,3072,2560)
reading predicted mask Inference_BEHOLD/688058_(1.00,24250,17044,8704,4096)_Vessel.png
reading true mask masksBEHOLD_corrected/688058_vessel_(1.00,24250,17044,8704,4096)-mask.png
calculating connected components
nlabels_predicted: 235.00
nlabels_true: 337.00
lookup_table.shape: (235, 337)
height: 4096 width: 8704
filling lookup table
TruePositives: 207
FalsePositives: 28
FalseNegatives: 0
calculate split or merged
split: 8
merge: 0

reading predicted mask Inference_BEHOLD/688058_(1.00,29154,33335,1024,2048)_Vessel.png
reading true mask masksBEHOLD_corrected/688058_vessel_(1.00,29154,33335,1024,2048)-mask.png
calculating connected components
nlabels_predicted: 13.00
nlabels_true: 22.00
lookup_table.shape: (13, 22)
height: 2048 width: 1024
filling lookup table
TruePositives: 13
FalsePositives: 0
FalseNegatives: 0
calculate split or merged
split: 2
merge: 0

reading predicted mask Inference_BEHOLD/688058_(1.00,41404,13179,1025,1025)_Ve